In [1]:
!rm -rf IFIC-SummerSchool-2025
!git clone https://github.com/M0V1/IFIC-SummerSchool-2025.git
!pip install  numpy pandas uproot matplotlib mplhep awkward

Cloning into 'IFIC-SummerSchool-2025'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.
DEPRECATION: Loading egg at /Users/morvan/.local/lib/python3.11/site-packages/classy-3.2.3-py3.11-macosx-11.1-arm64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [ ]:
import uproot
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
from concurrent.futures import ProcessPoolExecutor, as_completed
import time
# Integrated luminosity in inverse picobarns
lumi = 36000.

# Fraction of events to process
fraction = 1



files = [
    "IFIC-SummerSchool-2025/Ntuples_tautau/ODEO_FEB2025_v0_4lep_mc_410470.PhPy8EG_A14_ttbar_hdamp258p75_nonallhad.4lep.root",
    "IFIC-SummerSchool-2025/Ntuples_tautau/ODEO_FEB2025_v0_4lep_mc_345121.PowhegPy8EG_NNLOPS_nnlo_30_ggH125_tautaulm15hp20.4lep.root",
    "IFIC-SummerSchool-2025/Ntuples_tautau/ODEO_FEB2025_v0_4lep_data15_periodE.4lep.root"
]
tree_name = "analysis"

def calc_weight(data):
    weight_list = (data["ScaleFactor_TAU"] * data["ScaleFactor_DiTauTRIGGER"] * data["ScaleFactor_PILEUP"] *
                   (data["ScaleFactor_BTAG"] * data["mcWeight"] / data["sum_of_weights"]) *
                   (data["xsec"] * data["filteff"] * data["kfac"] * lumi))
    return weight_list

#def cut_trig(trigDT):
 #   return trigDT

#def Matched_cut(Matched):
   # return ak.sum(Matched == True, axis=1) >= 1

#def two_lep(tau_n):
    #return tau_n == 2

def cut_tau_pt(tau_pt):
    return ak.sum(tau_pt > 20, axis=1) >= 2

#def cut_met_et(met_et):
 #   return met_et <= 80

#def cut_charge(charge):
 #   return ak.sum(charge, axis=1) == 0

#def cut_type(type1):
   # return ak.sum(type1 == 15, axis=1) == 2  # 15 is the PDG ID for taus

#def ID_iso_cut(ID, iso):
 #   return (ak.sum(ID, axis=1) > 1) & (ak.sum(iso, axis=1) > 1)

#def cut_jet(E, pt, b_tag):
#    E_check = ak.sum(E >= 30, axis=1) >= 2
 #   pt_check = ak.sum(pt >= 30, axis=1) >= 2
#    btag_check = ak.sum(b_tag >= 3, axis=1) < 1
 #   return (pt_check) & (E_check) & (btag_check)

#def cut_DR(eta, phi, eta_M, phi_M):
 #   Dphi1 = np.arccos(np.cos(phi - phi_M[:, 0]))
 #   Dphi2 = np.arccos(np.cos(phi - phi_M[:, 1]))

  #  Deta1 = eta - eta_M[:, 0]
  #  Deta2 = eta - eta_M[:, 1]

  #  DR_M1 = np.sqrt((Deta1)**2 + (Dphi1)**2)
  #  DR_M2 = np.sqrt((Deta2)**2 + (Dphi2)**2)

  #  DR_M1_check = ak.sum(DR_M1 >= 0.4, axis=1) >= 1
  #  DR_M2_check = ak.sum(DR_M2 >= 0.4, axis=1) >= 1

  #  return (DR_M1_check) & (DR_M2_check)

#def cut_VBF(jet_E, pt, eta, phi):
 #   jet_px = pt * np.cos(phi)
  #  jet_py = pt * np.sin(phi)
   # jet_pz = pt / np.tan(2.0 * np.arctan(np.exp(-eta)))

   # def combo(list_1):
   #     jets_pairs = ak.combinations(list_1, 2, fields=['List1', 'List2'])
   #     sum_List = jets_pairs['List1'] + jets_pairs['List2']
   #     return sum_List

#    combo_jet_E = combo(jet_E)
#    combo_jet_px = combo(jet_px)
#    combo_jet_py = combo(jet_py)
#    combo_jet_pz = combo(jet_pz)#

#    Mass = np.sqrt(combo_jet_E**2 - (combo_jet_px**2 + combo_jet_py**2 + combo_jet_pz**2))
#
#   jets_pairs = ak.combinations(eta, 2, fields=['List1', 'List2'])
#    abs_dif_eta = np.abs(jets_pairs['List1'] - jets_pairs['List2'])
#    eta_mult = jets_pairs['List1'] * jets_pairs['List2']

#    return ak.sum((abs_dif_eta > 3) & (eta_mult < 0) & (Mass >= 500), axis=1) > 0

def Hmass(E, pt, eta, phi):
    tau_E = ak.sum(E, axis=1)
    tau_px = ak.sum(pt * np.cos(phi), axis=1)
    tau_py = ak.sum(pt * np.sin(phi), axis=1)
    tau_pz = ak.sum(pt / np.tan(2.0 * np.arctan(np.exp(-eta))), axis=1)
    Mass = np.sqrt(tau_E**2 - (tau_px**2 + tau_py**2 + tau_pz**2))
    return Mass

def process_file(file_path, tree_name, loop):
    tree = uproot.open(f"{file_path}:{tree_name}")
    sample_data = []

    variables = ["tau_pt", "tau_eta", "tau_phi", "tau_e", "tau_charge", "trigT", "lep_isTrigMatched", "tau_n", "met", "lep_type", "lep_isMediumID", "lep_isLooseIso", "jet_e", "jet_pt", "jet_eta", "jet_phi", "jet_btag_quantile", "ScaleFactor_TAU", "ScaleFactor_DiTauTRIGGER", "ScaleFactor_PILEUP", "ScaleFactor_BTAG", "mcWeight", "sum_of_weights", "xsec", "filteff", "kfac"]

    for data in tree.iterate(variables, library="ak", entry_start=int(tree.num_entries * fraction * loop), entry_stop=int(tree.num_entries * fraction * (loop + 1))):
      #  data = data[cut_trig(data.trigT)]
       # data = data[Matched_cut(data.lep_isTrigMatched)]
        #data = data[two_lep(data.tau_n)]
        #data = data[cut_met_et(data.met)]
       # data = data[cut_type(data.lep_type)]
        data = data[cut_tau_pt(data.tau_pt)]
       # data = data[cut_charge(data.tau_charge)]
        #data = data[ID_iso_cut(data.lep_isMediumID, data.lep_isLooseIso)]
       # data = data[cut_jet(data.jet_e, data.jet_pt, data.jet_btag_quantile)]
       # data = data[cut_DR(data.jet_eta, data.jet_phi, data.tau_eta, data.tau_phi)]
        #data = data[cut_VBF(data.jet_e, data.jet_pt, data.jet_eta, data.jet_phi)]

        data['Inv_mass'] = Hmass(data.tau_e, data.tau_pt, data.tau_eta, data.tau_phi)
        data['Weight'] = calc_weight(data)

        sample_data.append(data)

    return ak.concatenate(sample_data, axis=0)

def parallel_analysis(file_path, tree_name):
    with ProcessPoolExecutor() as executor:
        futures = {executor.submit(process_file, file_path, tree_name, i): i for i in range(10)}
        results = []
        for future in as_completed(futures):
            try:
                results.append(future.result())
            except Exception as e:
                print(f"Error in loop {futures[future]}: {e}")

    combined_array = ak.concatenate(results, axis=0) if results else ak.Array([])
    return combined_array

# Main analysis loop
start_all = time.time()
fraction = 1
print("The analysis has started")
bkg_data = parallel_analysis(files[0], tree_name)
signal_data = parallel_analysis(files[1], tree_name)
data_data = parallel_analysis(files[2], tree_name)

# Fonction de tracé mise à jour
def plot_data(bkg, signal, data, fit=True):
    xmin, xmax, step_size = 110, 160, 2
    bin_edges = np.arange(xmin, xmax + step_size, step_size)
    bin_centres = (bin_edges[:-1] + bin_edges[1:]) / 2

    # Histogramme des données (pas de poids)
    data_y, _ = np.histogram(data['Inv_mass'], bins=bin_edges)
    data_yerr = np.sqrt(data_y)

    # Histogrammes MC
    ttbar_y, _ = np.histogram(bkg['Inv_mass'], bins=bin_edges, weights=ttbar['Weight'])
    signal_y, _ = np.histogram(signal['Inv_mass'], bins=bin_edges, weights=signal['Weight'])

    # Empilement des MC
    stacked_mc = np.vstack([bkg_y, signal_y])
    total_mc = np.sum(stacked_mc, axis=0)

    # Incertitude statistique sur MC (ajout quadratique)
    total_mc_err = np.sqrt(
        np.histogram(ttbar['Inv_mass'], bins=bin_edges, weights=ttbar['Weight']**2)[0] +
        np.histogram(signal['Inv_mass'], bins=bin_edges, weights=signal['Weight']**2)[0]
    )

    fig, (main_ax, ratio_ax) = plt.subplots(2, 1, figsize=(7, 6), gridspec_kw={'height_ratios': [3, 1]}, sharex=True)

    # Tracé des données
    main_ax.errorbar(bin_centres, data_y, yerr=data_yerr, fmt='ko', label=f'Data ({sum(data_y)} entries)')

    # Tracé MC empilé
    main_ax.hist([bkg['Inv_mass'], signal['Inv_mass']],
                 bins=bin_edges,
                 weights=[bkg['Weight'], signal['Weight']],
                 stacked=True,
                 color=['cyan', 'red'],
                 label=[r'$t\bar{t}$', r'$ggH\to\tau\tau$'])

    # Barres d'incertitude sur le MC
    main_ax.bar(bin_centres, 2 * total_mc_err, alpha=0.5, bottom=total_mc - total_mc_err,
                color='none', hatch="////", width=step_size, label='Stat. unc.')

    main_ax.set_xlim(xmin, xmax)
    main_ax.set_ylim(0, max(max(data_y), max(total_mc)) * 1.5)
    main_ax.set_ylabel("Events")
    main_ax.text(0.05, 0.93, 'ATLAS Open Data', transform=main_ax.transAxes, fontsize=13)
    main_ax.text(0.05, 0.88, 'for education', transform=main_ax.transAxes, style='italic', fontsize=8)
    main_ax.text(0.05, 0.82, r'$\sqrt{s}=13$ TeV, 36 fb$^{-1}$', transform=main_ax.transAxes)
    main_ax.legend(frameon=False)

    # Plot ratio (Data / MC)
    ratio = data_y / total_mc
    ratio_err = ratio * (data_yerr / data_y)
    ratio_ax.errorbar(bin_centres, ratio, yerr=ratio_err, fmt='ko')
    ratio_ax.axhline(1, color='red', linestyle='--')
    ratio_ax.set_ylabel("Data / MC")
    ratio_ax.set_ylim(0.5, 1.5)
    ratio_ax.set_xlabel(r"$m_{\tau\tau}$ [GeV]")

    plt.tight_layout()
    plt.subplots_adjust(hspace=0.05)
    plt.show()

# Appel final
plot_data( bkg_data, signal_data, data_data)